template.csv predicate mapping

In [37]:
# === Dark Pattern predicate labeling (API-forced mapping, single-pass) ===
# Prereq: pip install openai python-dotenv pandas
# Uses: .env -> OPENAI_API_KEY
import os, re, json, math
import pandas as pd
from dotenv import load_dotenv

# =========================
# Settings
# =========================
load_dotenv()  # read OPENAI_API_KEY from .env
INPUT_CSV = "../data/0923preprocessed/template_full.csv"         # <- input 변경 가능
MODEL     = "gpt-4o"                                            # default model
OUT_PATH  = os.path.join(os.path.dirname(INPUT_CSV), "template_predicate.csv")  # <- output 경로

INCLUDE_DEFINITIONS = True
PRINT_DEBUG = True
N_DEBUG_ROWS = 3205

# (선택) 알려진 오타 보정
STRICT_TYPO_MAP = {
    "Pressured Seliing": "Pressured Selling"
}

# =========================
# Taxonomy
# =========================
CANONICAL_TYPES = ["Urgency","Misdirection","Social Proof","Scarcity","Not Dark Pattern"]

PREDICATES = {
    "Urgency":       ["Countdown Timers", "Limited-time Messages"],
    "Misdirection":  ["Confirmshaming", "Trick Questions", "Pressured Selling"],
    "Social Proof":  ["Activity Notifications", "Testimonials of Uncertain Origin"],
    "Scarcity":      ["Low-stock Messages", "High-demand Messages"],
    "Not Dark Pattern": ["None"],
}

# =========================
# Type & Predicate Definitions (from your spec)
# =========================
DEFINITIONS = {
    "Urgency": {
        "Definition": (
            "When users are under time pressure, they have less capacity to critically evaluate information and may "
            "feel anxiety or stress. Providers can exploit this to push users into actions not fully in their interest."
        ),
        "Predicates": {
            "Countdown Timers": "A visible countdown timer indicating an offer will end soon (explicit ticking timer).",
            "Limited-time Messages": "Claims that a deal will end soon/today without specifying a clear deadline."
        },
    },
    "Misdirection": {
        "Definition": (
            "Manipulates user attention (visual emphasis, wording tricks) to distract/redirect and downplay or hide alternatives, steering users toward "
            "unintended choices (diverting attention and inducing actions different from the user's original intent)."
        ),
        "Predicates": {
            "Confirmshaming": (
                "Shame/guilt framed especially on decline/opt-out labels to discourage opting out and nudge acceptance "
                "(e.g., 'No thanks, I hate saving money')."
            ),
            "Trick Questions": (
                "Ambiguous or double-negative wording that exploits scan reading and misleads users into unintended consent."
            ),
            "Pressured Selling": (
                "Preselected or prominently steered higher-priced variants/add-ons that pressure users to pay more."
            ),
        },
    },
    "Social Proof": {
        "Definition": (
            "Creates an illusion of popularity/credibility by presenting real or simulated social signals (reviews, "
            "testimonials, activity messages like 'N viewing/bought'), often exaggerated or fabricated, to exploit the "
            "social-proof bias and nudge conformity."
        ),
        "Predicates": {
            "Activity Notifications": (
                "Real or simulated activity signals (e.g., '6 people just bought', '120 viewing now'); may be exaggerated/fabricated."
            ),
            "Testimonials of Uncertain Origin": (
                "Testimonials/reviews/ratings whose origin or verification is unclear or unverifiable."
            ),
        },
    },
    "Scarcity": {
        "Definition": (
            "Creates an artificial or exaggerated sense of limited availability (misleading low stock or high demand), "
            "triggering FOMO and pushing hasty decisions without full evaluation."
        ),
        "Predicates": {
            "Low-stock Messages": "Warnings like 'Only 2 left in stock', often exaggerated/misleading to create scarcity.",
            "High-demand Messages": (
                "Signals of high demand or rapid sales (e.g., 'Selling fast', 'Booked 120 times today', '1,200 purchased in the last 24h')."
            ),
        },
    },
    "Not Dark Pattern": {
        "Definition": "Content that does not represent any dark pattern in this taxonomy.",
        "Predicates": {"None": "Always used for this Type."},
    },
}

# =========================
# SYSTEM PROMPT (강한 규칙 + 경계 규칙)
# =========================
SYSTEM_PROMPT = (
    "ROLE: Dark-Pattern Expert Annotator.\n"
    "Task: For the given text and the FIXED Type, choose exactly ONE predicate from the allowed list for that Type.\n"
    "Rules:\n"
    "- Do NOT change the Type.\n"
    "- If Type is 'Not Dark Pattern', predicate must be 'None'.\n"
    "- Keep rationale <= 120 chars. Return ONLY a compact JSON object with keys: predicate, confidence, rationale.\n"
    "- Base your choice strictly on the definitions and these disambiguation rules:\n"
    "  * Urgency: explicit ticking timer → Countdown Timers; time pressure w/o clear deadline → Limited-time Messages.\n"
    "  * Social Proof: real-time activity signals (viewing/bought now) → Activity Notifications; unclear-source endorsements → Testimonials of Uncertain Origin.\n"
    "  * Scarcity: explicit quantity/stock (Only N left) → Low-stock Messages; popularity/sales speed (selling fast, booked N times) → High-demand Messages.\n"
    "  * Misdirection: shame/guilt on opt-out → Confirmshaming; ambiguous/double-negative wording → Trick Questions; preselected/steered expensive options → Pressured Selling.\n"
)

# =========================
# Prompt builder
# =========================
def build_user_prompt(type_name: str, text: str) -> str:
    lines = []
    if INCLUDE_DEFINITIONS:
        info = DEFINITIONS.get(type_name, {})
        if "Definition" in info:
            lines.append(f"- Type definition: {info['Definition']}")
        preds = info.get("Predicates", {})
        if preds:
            lines.append("- Predicate definitions:")
            for k, v in preds.items():
                lines.append(f"  * {k}: {v}")

    allowed_list = "\n".join(f"- {p}" for p in PREDICATES[type_name])
    return f"""Assign exactly ONE predicate for the text, constrained by the given Type.

Type: {type_name}

{'\n'.join(lines) if lines else ''}

Allowed predicates (choose exactly ONE; return the exact string):
{allowed_list}

Text:
\"\"\"{text}\"\"\"

Return JSON only:
{{"predicate": "...", "confidence": 0.0-1.0, "rationale": "..."}}"""

# =========================
# API call (단 1회) + 최종 강제 클램프
# =========================
def _clamp_to_allowed(d: dict, allowed: list[str]) -> dict:
    """미허용/오타/비정형 응답을 최종적으로 허용 enum으로 강제."""
    pred = str(d.get("predicate", "")).strip()
    if pred in STRICT_TYPO_MAP:
        pred = STRICT_TYPO_MAP[pred]

    # confidence 정규화
    try:
        conf = float(d.get("confidence", 0.0))
        if math.isnan(conf): conf = 0.0
    except Exception:
        conf = 0.0
    conf = max(0.0, min(1.0, conf))

    rat = str(d.get("rationale", "")).strip()[:120] or "No rationale."

    # 🔒 핵심: 허용 enum에 없으면 타입별 첫 항목으로 강제
    if pred not in allowed:
        pred = allowed[0]
        rat = (rat + " [clamped to allowed]").strip()[:120]

    return {"predicate": pred, "confidence": conf, "rationale": rat}

def call_openai(type_name: str, text: str, model: str) -> dict:
    """
    단일 호출 + 최종 클램프: 어떤 경우에도 해당 Type의 enum 중 하나로 반환.
    """
    from openai import OpenAI
    client = OpenAI()

    allowed = PREDICATES[type_name]
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(type_name, text)},
    ]

    json_schema = {
        "type": "object",
        "properties": {
            "predicate": {"type": "string", "enum": allowed},
            "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "rationale": {"type": "string", "maxLength": 120},
        },
        "required": ["predicate", "confidence", "rationale"],
        "additionalProperties": False,
    }

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        tools=[{"type":"function","function":{"name":"set_predicate","description":"label","parameters":json_schema}}],
        tool_choice={"type":"function","function":{"name":"set_predicate"}},
    )
    msg = resp.choices[0].message

    if getattr(msg, "tool_calls", None):
        args = json.loads(msg.tool_calls[0].function.arguments)
        return _clamp_to_allowed(args, allowed)

    # 드문 fallback: content JSON
    text_out = (msg.content or "").strip()
    if text_out:
        try:
            data = json.loads(re.sub(r"^```(?:json)?\s*|\s*```$", "", text_out, flags=re.I|re.M))
            return _clamp_to_allowed(data, allowed)
        except Exception:
            pass

    # 완전 비정형이면 첫 항목으로
    return {"predicate": allowed[0], "confidence": 0.0, "rationale": "Model did not return structured output. [clamped]"}

# =========================
# Main
# =========================
df = pd.read_csv(INPUT_CSV)

# 기본 전처리
for col in ["String","Type"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Type 안전 필터
df = df[df["Type"].isin(CANONICAL_TYPES)].copy()
df = df.reset_index(drop=True)

# label 컬럼 없으면 신설
if "label" not in df.columns:
    df["label"] = ""

# 매핑
results = []
cache = {}
for i, row in df.iterrows():
    t = row["Type"]
    s = row["String"]

    if t == "Not Dark Pattern":
        res = {"predicate": "None", "confidence": 1.0, "rationale": "Type is 'Not Dark Pattern'; predicate must be 'None'."}
        results.append(res); cache[(t, s)] = res; continue

    key = (t, s)
    if key in cache:
        results.append(cache[key]); continue

    res = call_openai(t, s, MODEL)  # 1회 호출 (+ 내부 클램프 보장)
    results.append(res); cache[key] = res

# 출력 CSV(요구된 4개 컬럼만)
out = df[["String","Type","label"]].copy()
out["predicate"] = [r["predicate"] for r in results]

# 안전 보정(추가 방어막): 비-ND에 'None'이 남아 있으면 재클램프
for t, allowed in PREDICATES.items():
    if t == "Not Dark Pattern": continue
    mask = (out["Type"] == t) & (out["predicate"].astype(str).str.strip().isin(["None", ""]))
    if mask.any():
        out.loc[mask, "predicate"] = allowed[0]

# 저장
os.makedirs(os.path.dirname(OUT_PATH) or ".", exist_ok=True)
out.to_csv(OUT_PATH, index=False, encoding="utf-8")

print(f"[OK] saved → {OUT_PATH}")

# (선택) 콘솔 샘플 근거 출력
if PRINT_DEBUG:
    print("\n--- Sample mapping rationales (from model) ---")
    shown = 0
    for i in range(len(results)):
        if shown >= N_DEBUG_ROWS: break
        r = results[i]
        text_snip = df.loc[i, "String"][:160].replace("\n", " ")
        print(f"[{df.loc[i,'Type']}] -> {r['predicate']} (conf={r['confidence']:.2f}) :: {r['rationale']}\n  Text: {text_snip}")
        shown += 1

# 검증(비-ND에서 None 금지)
bad = out[(out["Type"] != "Not Dark Pattern") & (out["predicate"] == "None")]
print("Non-ND with 'None' rows (should be 0):", len(bad))


[OK] saved → ../data/0923preprocessed/template_predicate.csv

--- Sample mapping rationales (from model) ---
[Social Proof] -> Activity Notifications (conf=0.90) :: The text '79 clicks today' suggests real-time activity, fitting Activity Notifications.
  Text: Seen on product page 79 clicks today
[Urgency] -> Limited-time Messages (conf=0.90) :: The text implies urgency without a specific deadline or ticking timer.
  Text: Flash Sale ends in SHOP NOW
[Social Proof] -> Activity Notifications (conf=0.95) :: The text '17 people have viewed this wine today' is a real-time activity signal, fitting Activity Notifications.
  Text: 17 people have viewed this wine today
[Not Dark Pattern] -> None (conf=1.00) :: Type is 'Not Dark Pattern'; predicate must be 'None'.
  Text: These cookies are usually set by our marketing and advertising partners. They may be used by them to build a profile of your interest and later show you relevan
[Not Dark Pattern] -> None (conf=1.00) :: Type is 'Not Dark Patte

In [48]:
conf = pd.Series([float(r["confidence"]) for r in results])
print(conf.value_counts())

1.00    1683
0.95    1041
0.90     339
0.80      72
0.70      58
0.00       4
0.60       2
0.85       1
Name: count, dtype: int64


In [50]:
import pandas as pd

df = pd.read_csv("/Users/soyoung/404DNF_AI/data/0923preprocessed/template_predicate.csv", keep_default_na=False)

# 1) predicate 컬럼에서 'None' 정확히 몇 개?
p_none = (df["predicate"].astype(str).str.strip() == "None").sum()
print("predicate == 'None':", p_none)

# 2) Type이 Not Dark Pattern 몇 개?
nd = (df["Type"] == "Not Dark Pattern").sum()
print("Not Dark Pattern rows:", nd)

# 3) String 컬럼에 'None'(대소문자 무시) 들어있는 행 몇 개? (참고용)
str_contains_none = df["String"].astype(str).str.contains(r"\bnone\b", case=False, na=False).sum()
print("String contains 'none' (any case):", str_contains_none)

# 4) predicate의 “이상치” 확인: 비-ND인데 predicate가 None/빈값인 경우
weird = df[(df["Type"] != "Not Dark Pattern") &
           (df["predicate"].astype(str).str.strip().isin(["None",""])) ]
print("Non-ND with predicate None/empty:", len(weird))
print(weird.head(5))

# 5) predicate 유니크 값 형태(공백/숨은문자 진단)
print(df["predicate"].astype(str).apply(repr).value_counts().head(10))


predicate == 'None': 1600
Not Dark Pattern rows: 1600
String contains 'none' (any case): 0
Non-ND with predicate None/empty: 0
Empty DataFrame
Columns: [String, Type, label, predicate]
Index: []
predicate
'None'                                1600
'Activity Notifications'               361
'Low-stock Messages'                   360
'Limited-time Messages'                240
'Countdown Timers'                     160
'Pressured Selling'                    157
'Confirmshaming'                       137
'Trick Questions'                      106
'High-demand Messages'                  40
'Testimonials of Uncertain Origin'      39
Name: count, dtype: int64


contextual.csv predicate mapping

In [46]:
# === Dark Pattern predicate labeling (API-forced mapping, single-pass) ===
# Prereq: pip install openai python-dotenv pandas
# Uses: .env -> OPENAI_API_KEY
import os, re, json, math
import pandas as pd
from dotenv import load_dotenv

# =========================
# Settings
# =========================
load_dotenv()  # read OPENAI_API_KEY from .env
INPUT_CSV = "../data/0923preprocessed/contextual_full.csv"         # <- input 변경 가능
MODEL     = "gpt-4o"                                            # default model
OUT_PATH  = os.path.join(os.path.dirname(INPUT_CSV), "contextual_predicate.csv")  # <- output 경로

INCLUDE_DEFINITIONS = True
PRINT_DEBUG = True
N_DEBUG_ROWS = 3205

# (선택) 알려진 오타 보정
STRICT_TYPO_MAP = {
    "Pressured Seliing": "Pressured Selling"
}

# =========================
# Taxonomy
# =========================
CANONICAL_TYPES = ["Urgency","Misdirection","Social Proof","Scarcity","Not Dark Pattern"]

PREDICATES = {
    "Urgency":       ["Countdown Timers", "Limited-time Messages"],
    "Misdirection":  ["Confirmshaming", "Trick Questions", "Pressured Selling"],
    "Social Proof":  ["Activity Notifications", "Testimonials of Uncertain Origin"],
    "Scarcity":      ["Low-stock Messages", "High-demand Messages"],
    "Not Dark Pattern": ["None"],
}

# =========================
# Type & Predicate Definitions (from your spec)
# =========================
DEFINITIONS = {
    "Urgency": {
        "Definition": (
            "When users are under time pressure, they have less capacity to critically evaluate information and may "
            "feel anxiety or stress. Providers can exploit this to push users into actions not fully in their interest."
        ),
        "Predicates": {
            "Countdown Timers": "A visible countdown timer indicating an offer will end soon (explicit ticking timer).",
            "Limited-time Messages": "Claims that a deal will end soon/today without specifying a clear deadline."
        },
    },
    "Misdirection": {
        "Definition": (
            "Manipulates user attention (visual emphasis, wording tricks) to distract/redirect and downplay or hide alternatives, steering users toward "
            "unintended choices (diverting attention and inducing actions different from the user's original intent)."
        ),
        "Predicates": {
            "Confirmshaming": (
                "Shame/guilt framed especially on decline/opt-out labels to discourage opting out and nudge acceptance "
                "(e.g., 'No thanks, I hate saving money')."
            ),
            "Trick Questions": (
                "Ambiguous or double-negative wording that exploits scan reading and misleads users into unintended consent."
            ),
            "Pressured Selling": (
                "Preselected or prominently steered higher-priced variants/add-ons that pressure users to pay more."
            ),
        },
    },
    "Social Proof": {
        "Definition": (
            "Creates an illusion of popularity/credibility by presenting real or simulated social signals (reviews, "
            "testimonials, activity messages like 'N viewing/bought'), often exaggerated or fabricated, to exploit the "
            "social-proof bias and nudge conformity."
        ),
        "Predicates": {
            "Activity Notifications": (
                "Real or simulated activity signals (e.g., '6 people just bought', '120 viewing now'); may be exaggerated/fabricated."
            ),
            "Testimonials of Uncertain Origin": (
                "Testimonials/reviews/ratings whose origin or verification is unclear or unverifiable."
            ),
        },
    },
    "Scarcity": {
        "Definition": (
            "Creates an artificial or exaggerated sense of limited availability (misleading low stock or high demand), "
            "triggering FOMO and pushing hasty decisions without full evaluation."
        ),
        "Predicates": {
            "Low-stock Messages": "Warnings like 'Only 2 left in stock', often exaggerated/misleading to create scarcity.",
            "High-demand Messages": (
                "Signals of high demand or rapid sales (e.g., 'Selling fast', 'Booked 120 times today', '1,200 purchased in the last 24h')."
            ),
        },
    },
    "Not Dark Pattern": {
        "Definition": "Content that does not represent any dark pattern in this taxonomy.",
        "Predicates": {"None": "Always used for this Type."},
    },
}

# =========================
# SYSTEM PROMPT (강한 규칙 + 경계 규칙)
# =========================
SYSTEM_PROMPT = (
    "ROLE: Dark-Pattern Expert Annotator.\n"
    "Task: For the given text and the FIXED Type, choose exactly ONE predicate from the allowed list for that Type.\n"
    "Rules:\n"
    "- Do NOT change the Type.\n"
    "- If Type is 'Not Dark Pattern', predicate must be 'None'.\n"
    "- Keep rationale <= 120 chars. Return ONLY a compact JSON object with keys: predicate, confidence, rationale.\n"
    "- Base your choice strictly on the definitions and these disambiguation rules:\n"
    "  * Urgency: explicit ticking timer → Countdown Timers; time pressure w/o clear deadline → Limited-time Messages.\n"
    "  * Social Proof: real-time activity signals (viewing/bought now) → Activity Notifications; unclear-source endorsements → Testimonials of Uncertain Origin.\n"
    "  * Scarcity: explicit quantity/stock (Only N left) → Low-stock Messages; popularity/sales speed (selling fast, booked N times) → High-demand Messages.\n"
    "  * Misdirection: shame/guilt on opt-out → Confirmshaming; ambiguous/double-negative wording → Trick Questions; preselected/steered expensive options → Pressured Selling.\n"
)

# =========================
# Prompt builder
# =========================
def build_user_prompt(type_name: str, text: str) -> str:
    lines = []
    if INCLUDE_DEFINITIONS:
        info = DEFINITIONS.get(type_name, {})
        if "Definition" in info:
            lines.append(f"- Type definition: {info['Definition']}")
        preds = info.get("Predicates", {})
        if preds:
            lines.append("- Predicate definitions:")
            for k, v in preds.items():
                lines.append(f"  * {k}: {v}")

    allowed_list = "\n".join(f"- {p}" for p in PREDICATES[type_name])
    return f"""Assign exactly ONE predicate for the text, constrained by the given Type.

Type: {type_name}

{'\n'.join(lines) if lines else ''}

Allowed predicates (choose exactly ONE; return the exact string):
{allowed_list}

Text:
\"\"\"{text}\"\"\"

Return JSON only:
{{"predicate": "...", "confidence": 0.0-1.0, "rationale": "..."}}"""

# =========================
# API call (단 1회) + 최종 강제 클램프
# =========================
def _clamp_to_allowed(d: dict, allowed: list[str]) -> dict:
    """미허용/오타/비정형 응답을 최종적으로 허용 enum으로 강제."""
    pred = str(d.get("predicate", "")).strip()
    if pred in STRICT_TYPO_MAP:
        pred = STRICT_TYPO_MAP[pred]

    # confidence 정규화
    try:
        conf = float(d.get("confidence", 0.0))
        if math.isnan(conf): conf = 0.0
    except Exception:
        conf = 0.0
    conf = max(0.0, min(1.0, conf))

    rat = str(d.get("rationale", "")).strip()[:120] or "No rationale."

    # 🔒 핵심: 허용 enum에 없으면 타입별 첫 항목으로 강제
    if pred not in allowed:
        pred = allowed[0]
        rat = (rat + " [clamped to allowed]").strip()[:120]

    return {"predicate": pred, "confidence": conf, "rationale": rat}

def call_openai(type_name: str, text: str, model: str) -> dict:
    """
    단일 호출 + 최종 클램프: 어떤 경우에도 해당 Type의 enum 중 하나로 반환.
    """
    from openai import OpenAI
    client = OpenAI()

    allowed = PREDICATES[type_name]
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(type_name, text)},
    ]

    json_schema = {
        "type": "object",
        "properties": {
            "predicate": {"type": "string", "enum": allowed},
            "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "rationale": {"type": "string", "maxLength": 120},
        },
        "required": ["predicate", "confidence", "rationale"],
        "additionalProperties": False,
    }

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        tools=[{"type":"function","function":{"name":"set_predicate","description":"label","parameters":json_schema}}],
        tool_choice={"type":"function","function":{"name":"set_predicate"}},
    )
    msg = resp.choices[0].message

    if getattr(msg, "tool_calls", None):
        args = json.loads(msg.tool_calls[0].function.arguments)
        return _clamp_to_allowed(args, allowed)

    # 드문 fallback: content JSON
    text_out = (msg.content or "").strip()
    if text_out:
        try:
            data = json.loads(re.sub(r"^```(?:json)?\s*|\s*```$", "", text_out, flags=re.I|re.M))
            return _clamp_to_allowed(data, allowed)
        except Exception:
            pass

    # 완전 비정형이면 첫 항목으로
    return {"predicate": allowed[0], "confidence": 0.0, "rationale": "Model did not return structured output. [clamped]"}

# =========================
# Main
# =========================
df = pd.read_csv(INPUT_CSV)

# 기본 전처리
for col in ["String","Type"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Type 안전 필터
df = df[df["Type"].isin(CANONICAL_TYPES)].copy()
df = df.reset_index(drop=True)

# label 컬럼 없으면 신설
if "label" not in df.columns:
    df["label"] = ""

# 매핑
results = []
cache = {}
for i, row in df.iterrows():
    t = row["Type"]
    s = row["String"]

    if t == "Not Dark Pattern":
        res = {"predicate": "None", "confidence": 1.0, "rationale": "Type is 'Not Dark Pattern'; predicate must be 'None'."}
        results.append(res); cache[(t, s)] = res; continue

    key = (t, s)
    if key in cache:
        results.append(cache[key]); continue

    res = call_openai(t, s, MODEL)  # 1회 호출 (+ 내부 클램프 보장)
    results.append(res); cache[key] = res

# 출력 CSV(요구된 4개 컬럼만)
out = df[["String","Type","label"]].copy()
out["predicate"] = [r["predicate"] for r in results]

# 안전 보정(추가 방어막): 비-ND에 'None'이 남아 있으면 재클램프
for t, allowed in PREDICATES.items():
    if t == "Not Dark Pattern": continue
    mask = (out["Type"] == t) & (out["predicate"].astype(str).str.strip().isin(["None", ""]))
    if mask.any():
        out.loc[mask, "predicate"] = allowed[0]

# 저장
os.makedirs(os.path.dirname(OUT_PATH) or ".", exist_ok=True)
out.to_csv(OUT_PATH, index=False, encoding="utf-8")

print(f"[OK] saved → {OUT_PATH}")

# (선택) 콘솔 샘플 근거 출력
if PRINT_DEBUG:
    print("\n--- Sample mapping rationales (from model) ---")
    shown = 0
    for i in range(len(results)):
        if shown >= N_DEBUG_ROWS: break
        r = results[i]
        text_snip = df.loc[i, "String"][:160].replace("\n", " ")
        print(f"[{df.loc[i,'Type']}] -> {r['predicate']} (conf={r['confidence']:.2f}) :: {r['rationale']}\n  Text: {text_snip}")
        shown += 1

# 검증(비-ND에서 None 금지)
bad = out[(out["Type"] != "Not Dark Pattern") & (out["predicate"] == "None")]
print("Non-ND with 'None' rows (should be 0):", len(bad))


[OK] saved → ../data/0923preprocessed/contextual_predicate.csv

--- Sample mapping rationales (from model) ---
[Social Proof] -> Activity Notifications (conf=0.95) :: The text '19 people buy this product per day' suggests real-time activity, fitting Activity Notifications.
  Text: 19 people buy this product per day
[Urgency] -> Limited-time Messages (conf=0.90) :: The text indicates urgency with 'Flash Sale ends' but lacks a specific deadline or countdown timer.
  Text: Flash Sale ends in SHOP NOW
[Social Proof] -> Activity Notifications (conf=0.95) :: The text '17 people have viewed this wine today' is a real-time activity signal, fitting Activity Notifications.
  Text: 17 people have viewed this wine today
[Not Dark Pattern] -> None (conf=1.00) :: Type is 'Not Dark Pattern'; predicate must be 'None'.
  Text: These cookies are usually set by our marketing and advertising partners. They may be used by them to build a profile of your interest and later show you relevan
[Not Dark Pattern

In [47]:
conf = pd.Series([float(r["confidence"]) for r in results])
print(conf.value_counts())

1.00    1683
0.95    1041
0.90     339
0.80      72
0.70      58
0.00       4
0.60       2
0.85       1
Name: count, dtype: int64


In [49]:
import pandas as pd

df = pd.read_csv("/Users/soyoung/404DNF_AI/data/0923preprocessed/contextual_predicate.csv", keep_default_na=False)

# 1) predicate 컬럼에서 'None' 정확히 몇 개?
p_none = (df["predicate"].astype(str).str.strip() == "None").sum()
print("predicate == 'None':", p_none)

# 2) Type이 Not Dark Pattern 몇 개?
nd = (df["Type"] == "Not Dark Pattern").sum()
print("Not Dark Pattern rows:", nd)

# 3) String 컬럼에 'None'(대소문자 무시) 들어있는 행 몇 개? (참고용)
str_contains_none = df["String"].astype(str).str.contains(r"\bnone\b", case=False, na=False).sum()
print("String contains 'none' (any case):", str_contains_none)

# 4) predicate의 “이상치” 확인: 비-ND인데 predicate가 None/빈값인 경우
weird = df[(df["Type"] != "Not Dark Pattern") &
           (df["predicate"].astype(str).str.strip().isin(["None",""])) ]
print("Non-ND with predicate None/empty:", len(weird))
print(weird.head(5))

# 5) predicate 유니크 값 형태(공백/숨은문자 진단)
print(df["predicate"].astype(str).apply(repr).value_counts().head(10))


predicate == 'None': 1600
Not Dark Pattern rows: 1600
String contains 'none' (any case): 0
Non-ND with predicate None/empty: 0
Empty DataFrame
Columns: [String, Type, label, predicate]
Index: []
predicate
'None'                                1600
'Activity Notifications'               393
'Low-stock Messages'                   360
'Confirmshaming'                       279
'Countdown Timers'                     213
'Limited-time Messages'                187
'Pressured Selling'                     74
'Trick Questions'                       47
'High-demand Messages'                  40
'Testimonials of Uncertain Origin'       7
Name: count, dtype: int64


paraphrase predicate mapping

In [41]:
# === Dark Pattern predicate labeling (API-forced mapping, single-pass) ===
# Prereq: pip install openai python-dotenv pandas
# Uses: .env -> OPENAI_API_KEY
import os, re, json, math
import pandas as pd
from dotenv import load_dotenv

# =========================
# Settings
# =========================
load_dotenv()  # read OPENAI_API_KEY from .env
INPUT_CSV = "../data/0923preprocessed/paraphrase_full.csv"         # <- input 변경 가능
MODEL     = "gpt-4o"                                            # default model
OUT_PATH  = os.path.join(os.path.dirname(INPUT_CSV), "paraphrase_predicate.csv")  # <- output 경로

INCLUDE_DEFINITIONS = True
PRINT_DEBUG = True
N_DEBUG_ROWS = 3205

# (선택) 알려진 오타 보정
STRICT_TYPO_MAP = {
    "Pressured Seliing": "Pressured Selling"
}

# =========================
# Taxonomy
# =========================
CANONICAL_TYPES = ["Urgency","Misdirection","Social Proof","Scarcity","Not Dark Pattern"]

PREDICATES = {
    "Urgency":       ["Countdown Timers", "Limited-time Messages"],
    "Misdirection":  ["Confirmshaming", "Trick Questions", "Pressured Selling"],
    "Social Proof":  ["Activity Notifications", "Testimonials of Uncertain Origin"],
    "Scarcity":      ["Low-stock Messages", "High-demand Messages"],
    "Not Dark Pattern": ["None"],
}

# =========================
# Type & Predicate Definitions (from your spec)
# =========================
DEFINITIONS = {
    "Urgency": {
        "Definition": (
            "When users are under time pressure, they have less capacity to critically evaluate information and may "
            "feel anxiety or stress. Providers can exploit this to push users into actions not fully in their interest."
        ),
        "Predicates": {
            "Countdown Timers": "A visible countdown timer indicating an offer will end soon (explicit ticking timer).",
            "Limited-time Messages": "Claims that a deal will end soon/today without specifying a clear deadline."
        },
    },
    "Misdirection": {
        "Definition": (
            "Manipulates user attention (visual emphasis, wording tricks) to distract/redirect and downplay or hide alternatives, steering users toward "
            "unintended choices (diverting attention and inducing actions different from the user's original intent)."
        ),
        "Predicates": {
            "Confirmshaming": (
                "Shame/guilt framed especially on decline/opt-out labels to discourage opting out and nudge acceptance "
                "(e.g., 'No thanks, I hate saving money')."
            ),
            "Trick Questions": (
                "Ambiguous or double-negative wording that exploits scan reading and misleads users into unintended consent."
            ),
            "Pressured Selling": (
                "Preselected or prominently steered higher-priced variants/add-ons that pressure users to pay more."
            ),
        },
    },
    "Social Proof": {
        "Definition": (
            "Creates an illusion of popularity/credibility by presenting real or simulated social signals (reviews, "
            "testimonials, activity messages like 'N viewing/bought'), often exaggerated or fabricated, to exploit the "
            "social-proof bias and nudge conformity."
        ),
        "Predicates": {
            "Activity Notifications": (
                "Real or simulated activity signals (e.g., '6 people just bought', '120 viewing now'); may be exaggerated/fabricated."
            ),
            "Testimonials of Uncertain Origin": (
                "Testimonials/reviews/ratings whose origin or verification is unclear or unverifiable."
            ),
        },
    },
    "Scarcity": {
        "Definition": (
            "Creates an artificial or exaggerated sense of limited availability (misleading low stock or high demand), "
            "triggering FOMO and pushing hasty decisions without full evaluation."
        ),
        "Predicates": {
            "Low-stock Messages": "Warnings like 'Only 2 left in stock', often exaggerated/misleading to create scarcity.",
            "High-demand Messages": (
                "Signals of high demand or rapid sales (e.g., 'Selling fast', 'Booked 120 times today', '1,200 purchased in the last 24h')."
            ),
        },
    },
    "Not Dark Pattern": {
        "Definition": "Content that does not represent any dark pattern in this taxonomy.",
        "Predicates": {"None": "Always used for this Type."},
    },
}

# =========================
# SYSTEM PROMPT (강한 규칙 + 경계 규칙)
# =========================
SYSTEM_PROMPT = (
    "ROLE: Dark-Pattern Expert Annotator.\n"
    "Task: For the given text and the FIXED Type, choose exactly ONE predicate from the allowed list for that Type.\n"
    "Rules:\n"
    "- Do NOT change the Type.\n"
    "- If Type is 'Not Dark Pattern', predicate must be 'None'.\n"
    "- Keep rationale <= 120 chars. Return ONLY a compact JSON object with keys: predicate, confidence, rationale.\n"
    "- Base your choice strictly on the definitions and these disambiguation rules:\n"
    "  * Urgency: explicit ticking timer → Countdown Timers; time pressure w/o clear deadline → Limited-time Messages.\n"
    "  * Social Proof: real-time activity signals (viewing/bought now) → Activity Notifications; unclear-source endorsements → Testimonials of Uncertain Origin.\n"
    "  * Scarcity: explicit quantity/stock (Only N left) → Low-stock Messages; popularity/sales speed (selling fast, booked N times) → High-demand Messages.\n"
    "  * Misdirection: shame/guilt on opt-out → Confirmshaming; ambiguous/double-negative wording → Trick Questions; preselected/steered expensive options → Pressured Selling.\n"
)

# =========================
# Prompt builder
# =========================
def build_user_prompt(type_name: str, text: str) -> str:
    lines = []
    if INCLUDE_DEFINITIONS:
        info = DEFINITIONS.get(type_name, {})
        if "Definition" in info:
            lines.append(f"- Type definition: {info['Definition']}")
        preds = info.get("Predicates", {})
        if preds:
            lines.append("- Predicate definitions:")
            for k, v in preds.items():
                lines.append(f"  * {k}: {v}")

    allowed_list = "\n".join(f"- {p}" for p in PREDICATES[type_name])
    return f"""Assign exactly ONE predicate for the text, constrained by the given Type.

Type: {type_name}

{'\n'.join(lines) if lines else ''}

Allowed predicates (choose exactly ONE; return the exact string):
{allowed_list}

Text:
\"\"\"{text}\"\"\"

Return JSON only:
{{"predicate": "...", "confidence": 0.0-1.0, "rationale": "..."}}"""

# =========================
# API call (단 1회) + 최종 강제 클램프
# =========================
def _clamp_to_allowed(d: dict, allowed: list[str]) -> dict:
    """미허용/오타/비정형 응답을 최종적으로 허용 enum으로 강제."""
    pred = str(d.get("predicate", "")).strip()
    if pred in STRICT_TYPO_MAP:
        pred = STRICT_TYPO_MAP[pred]

    # confidence 정규화
    try:
        conf = float(d.get("confidence", 0.0))
        if math.isnan(conf): conf = 0.0
    except Exception:
        conf = 0.0
    conf = max(0.0, min(1.0, conf))

    rat = str(d.get("rationale", "")).strip()[:120] or "No rationale."

    # 🔒 핵심: 허용 enum에 없으면 타입별 첫 항목으로 강제
    if pred not in allowed:
        pred = allowed[0]
        rat = (rat + " [clamped to allowed]").strip()[:120]

    return {"predicate": pred, "confidence": conf, "rationale": rat}

def call_openai(type_name: str, text: str, model: str) -> dict:
    """
    단일 호출 + 최종 클램프: 어떤 경우에도 해당 Type의 enum 중 하나로 반환.
    """
    from openai import OpenAI
    client = OpenAI()

    allowed = PREDICATES[type_name]
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(type_name, text)},
    ]

    json_schema = {
        "type": "object",
        "properties": {
            "predicate": {"type": "string", "enum": allowed},
            "confidence": {"type": "number", "minimum": 0.0, "maximum": 1.0},
            "rationale": {"type": "string", "maxLength": 120},
        },
        "required": ["predicate", "confidence", "rationale"],
        "additionalProperties": False,
    }

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        tools=[{"type":"function","function":{"name":"set_predicate","description":"label","parameters":json_schema}}],
        tool_choice={"type":"function","function":{"name":"set_predicate"}},
    )
    msg = resp.choices[0].message

    if getattr(msg, "tool_calls", None):
        args = json.loads(msg.tool_calls[0].function.arguments)
        return _clamp_to_allowed(args, allowed)

    # 드문 fallback: content JSON
    text_out = (msg.content or "").strip()
    if text_out:
        try:
            data = json.loads(re.sub(r"^```(?:json)?\s*|\s*```$", "", text_out, flags=re.I|re.M))
            return _clamp_to_allowed(data, allowed)
        except Exception:
            pass

    # 완전 비정형이면 첫 항목으로
    return {"predicate": allowed[0], "confidence": 0.0, "rationale": "Model did not return structured output. [clamped]"}

# =========================
# Main
# =========================
df = pd.read_csv(INPUT_CSV)

# 기본 전처리
for col in ["String","Type"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Type 안전 필터
df = df[df["Type"].isin(CANONICAL_TYPES)].copy()
df = df.reset_index(drop=True)

# label 컬럼 없으면 신설
if "label" not in df.columns:
    df["label"] = ""

# 매핑
results = []
cache = {}
for i, row in df.iterrows():
    t = row["Type"]
    s = row["String"]

    if t == "Not Dark Pattern":
        res = {"predicate": "None", "confidence": 1.0, "rationale": "Type is 'Not Dark Pattern'; predicate must be 'None'."}
        results.append(res); cache[(t, s)] = res; continue

    key = (t, s)
    if key in cache:
        results.append(cache[key]); continue

    res = call_openai(t, s, MODEL)  # 1회 호출 (+ 내부 클램프 보장)
    results.append(res); cache[key] = res

# 출력 CSV(요구된 4개 컬럼만)
out = df[["String","Type","label"]].copy()
out["predicate"] = [r["predicate"] for r in results]

# 안전 보정(추가 방어막): 비-ND에 'None'이 남아 있으면 재클램프
for t, allowed in PREDICATES.items():
    if t == "Not Dark Pattern": continue
    mask = (out["Type"] == t) & (out["predicate"].astype(str).str.strip().isin(["None", ""]))
    if mask.any():
        out.loc[mask, "predicate"] = allowed[0]

# 저장
os.makedirs(os.path.dirname(OUT_PATH) or ".", exist_ok=True)
out.to_csv(OUT_PATH, index=False, encoding="utf-8")

print(f"[OK] saved → {OUT_PATH}")

# (선택) 콘솔 샘플 근거 출력
if PRINT_DEBUG:
    print("\n--- Sample mapping rationales (from model) ---")
    shown = 0
    for i in range(len(results)):
        if shown >= N_DEBUG_ROWS: break
        r = results[i]
        text_snip = df.loc[i, "String"][:160].replace("\n", " ")
        print(f"[{df.loc[i,'Type']}] -> {r['predicate']} (conf={r['confidence']:.2f}) :: {r['rationale']}\n  Text: {text_snip}")
        shown += 1

# 검증(비-ND에서 None 금지)
bad = out[(out["Type"] != "Not Dark Pattern") & (out["predicate"] == "None")]
print("Non-ND with 'None' rows (should be 0):", len(bad))
         

[OK] saved → ../data/0923preprocessed/paraphrase_predicate.csv

--- Sample mapping rationales (from model) ---
[Social Proof] -> Activity Notifications (conf=0.95) :: The text '1 person is looking at this item' is a real-time activity signal, fitting Activity Notifications.
  Text: 1 person is looking at this item.
[Urgency] -> Limited-time Messages (conf=0.90) :: The text indicates urgency with 'Flash Sale ends' but lacks a specific deadline or countdown timer.
  Text: Flash Sale ends in SHOP NOW
[Social Proof] -> Activity Notifications (conf=0.95) :: The text '17 people have viewed this wine today' is a real-time activity signal.
  Text: 17 people have viewed this wine today
[Not Dark Pattern] -> None (conf=1.00) :: Type is 'Not Dark Pattern'; predicate must be 'None'.
  Text: These cookies are usually set by our marketing and advertising partners. They may be used by them to build a profile of your interest and later show you relevan
[Not Dark Pattern] -> None (conf=1.00) :: Type is

In [44]:
conf = pd.Series([float(r["confidence"]) for r in results])
print(conf.value_counts())

1.00    1688
0.95    1134
0.90     246
0.70      65
0.80      65
0.00       1
0.85       1
Name: count, dtype: int64


In [45]:
import pandas as pd

df = pd.read_csv("/Users/soyoung/404DNF_AI/data/0923preprocessed/paraphrase_predicate.csv", keep_default_na=False)

# 1) predicate 컬럼에서 'None' 정확히 몇 개?
p_none = (df["predicate"].astype(str).str.strip() == "None").sum()
print("predicate == 'None':", p_none)

# 2) Type이 Not Dark Pattern 몇 개?
nd = (df["Type"] == "Not Dark Pattern").sum()
print("Not Dark Pattern rows:", nd)

# 3) String 컬럼에 'None'(대소문자 무시) 들어있는 행 몇 개? (참고용)
str_contains_none = df["String"].astype(str).str.contains(r"\bnone\b", case=False, na=False).sum()
print("String contains 'none' (any case):", str_contains_none)

# 4) predicate의 “이상치” 확인: 비-ND인데 predicate가 None/빈값인 경우
weird = df[(df["Type"] != "Not Dark Pattern") &
           (df["predicate"].astype(str).str.strip().isin(["None",""])) ]
print("Non-ND with predicate None/empty:", len(weird))
print(weird.head(5))

# 5) predicate 유니크 값 형태(공백/숨은문자 진단)
print(df["predicate"].astype(str).apply(repr).value_counts().head(10))


predicate == 'None': 1600
Not Dark Pattern rows: 1600
String contains 'none' (any case): 0
Non-ND with predicate None/empty: 0
Empty DataFrame
Columns: [String, Type, label, predicate]
Index: []
predicate
'None'                                1600
'Activity Notifications'               394
'Low-stock Messages'                   360
'Confirmshaming'                       273
'Countdown Timers'                     238
'Limited-time Messages'                162
'Pressured Selling'                     84
'Trick Questions'                       43
'High-demand Messages'                  40
'Testimonials of Uncertain Origin'       6
Name: count, dtype: int64
